In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.offline as pyo
import plotly.graph_objs as go
import dash
import dash_core_components as dcc
import dash_html_components as html
import seaborn as sns
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage,fcluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_curve, auc, adjusted_rand_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import svm
from scipy.stats import pearsonr, spearmanr


<ipython-input-1-b3d3bdcee481>:7: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

<ipython-input-1-b3d3bdcee481>:8: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In [2]:
import pandas as pd

# Read the CSV files
commodity = pd.read_csv('commodity_f.csv',sep = ',')

In [3]:
commodity

,Country,commodity,flow,trade_usd,quantity_name,quantity,category,Carbon Footprint
0,Afghanistan,"Sheep, live",Export,6088.000,Number of items,51.00,01_live_animals,0.18
1,Afghanistan,"Goats, live",Export,3958.000,Number of items,53.00,01_live_animals,0.18
2,Albania,"Bovine animals, live, except pure-bred breeding",Import,2414533.000,Number of items,6853.00,01_live_animals,0.87
3,Albania,"Swine, live except pure-bred breeding > 50 kg",Import,7648728.875,Number of items,96040.00,01_live_animals,0.87
4,Albania,"Fowls, live domestic < 185 grams",Import,2671732.000,Number of items,4895775.75,01_live_animals,0.87
...,...,...,...,...,...,...,...,...
37403,Madagascar,"Seaweeds and other algae,",Export,1014203.000,Weight in kilograms,1687717.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37404,Madagascar,Vegetable products nes for human consumption,Import,4137.000,Weight in kilograms,1369.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37405,Madagascar,"Cereal straw & husks, unprepared",Export,11.000,Weight in kilograms,4.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37406,Madagascar,Lucerne (alfalfa) meal and pellets,Import,1720.000,Weight in kilograms,2000.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07


In [4]:
commodity.dtypes

Country              object
commodity            object
flow                 object
trade_usd           float64
quantity_name        object
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [5]:
# Séparer les features et la variable cible
X = commodity[[ 'Country', 'commodity', 'flow', 'trade_usd', 'quantity_name', 'quantity', 'Carbon Footprint']]
y = commodity['category']

In [6]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd# Convertir les variables catégorielles en entiers
label_encoder = LabelEncoder()
commodity['Country'] = label_encoder.fit_transform(commodity['Country'])
commodity['commodity'] = label_encoder.fit_transform(commodity['commodity'])
commodity['flow'] = label_encoder.fit_transform(commodity['flow'])
commodity['quantity_name'] = label_encoder.fit_transform(commodity['quantity_name'])
# Séparer la variable cible des variables d'entrée
X = commodity.drop('category', axis=1)
y = commodity['category']

In [7]:
commodity.dtypes

Country               int32
commodity             int32
flow                  int32
trade_usd           float64
quantity_name         int32
quantity            float64
category             object
Carbon Footprint    float64
dtype: object

In [8]:
commodity

,Country,commodity,flow,trade_usd,quantity_name,quantity,category,Carbon Footprint
0,0,352,0,6088.000,1,51.00,01_live_animals,0.18
1,0,170,0,3958.000,1,53.00,01_live_animals,0.18
2,1,30,1,2414533.000,1,6853.00,01_live_animals,0.87
3,1,385,1,7648728.875,1,96040.00,01_live_animals,0.87
4,1,156,1,2671732.000,1,4895775.75,01_live_animals,0.87
...,...,...,...,...,...,...,...,...
37403,64,331,0,1014203.000,2,1687717.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37404,64,409,1,4137.000,2,1369.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37405,64,66,0,11.000,2,4.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07
37406,64,219,1,1720.000,2,2000.00,12_oil_seed_oleagic_fruits_grain_seed_fruit_et...,0.07


In [9]:
# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Créer un modèle d'arbre de décision
decision_tree = DecisionTreeClassifier()

# Entraîner le modèle sur les données d'entraînement
decision_tree.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred_tree = decision_tree.predict(X_test)

# Calculer la précision du modèle
accuracy = accuracy_score(y_test, y_pred_tree)
print("Précision : %.2f%%" % (accuracy * 100.0))

Précision : 99.80%


In [10]:
com_svm=commodity

In [11]:
label_encoder = LabelEncoder()
com_svm['category'] = label_encoder.fit_transform(com_svm['category'])

In [12]:
# Séparer la variable cible des variables d'entrée
X1 = com_svm.drop('category', axis=1)
y1 = com_svm['category']

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Créer un objet RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle
random_forest.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_forest = random_forest.predict(X_test)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred_forest)
print("Accuracy : {:.2f}%".format(accuracy*100))

Accuracy : 66.03%


In [14]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Créer un objet KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# Entraîner le modèle
knn.fit(X_train, y_train)

# Faire des prédictions sur l'ensemble de test
y_pred_knn = knn.predict(X_test)

# Calculer l'accuracy du modèle
accuracy = accuracy_score(y_test, y_pred_knn)
print("Accuracy : {:.2f}%".format(accuracy*100))

Accuracy : 25.02%


C:\Users\zeffz\anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:211: FutureWarning:

Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.



In [15]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.multiclass import OneVsRestClassifier
# Appliquer la méthode OVA pour résoudre le problème multiclasse
ova_model1 = OneVsRestClassifier(decision_tree)
ova_model1.fit(X_train, y_train)
ova_model2 = OneVsRestClassifier(random_forest)
ova_model2.fit(X_train, y_train)
ova_model3 = OneVsRestClassifier(knn)
ova_model3.fit(X_train, y_train)
# Faire des prédictions sur le jeu de test
y_pred_tree = ova_model1.predict(X_test)
y_pred_forest = ova_model2.predict(X_test)
y_pred_knn = ova_model3.predict(X_test)


In [16]:
# Calculer les scores de prédiction pour chaque modèle
y_pred_tree = decision_tree.predict_proba(X_test)[:, 1]
y_pred_forest = random_forest.predict_proba(X_test)[:, 1]
y_pred_knn = knn.predict_proba(X_test)[:, 1]

# Calculer les taux de faux positifs et vrais positifs pour chaque modèle
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_pred_tree)
fpr_forest, tpr_forest, _ = roc_curve(y_test, y_pred_forest)
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred_knn)

# Compute area under the curve (AUC)
auc_tree = auc(fpr_tree, tpr_tree)
auc_forest = auc(fpr_forest, tpr_forest)
auc_knn = auc(fpr_knn, tpr_knn)

# Plot ROC curve
plt.plot(fpr_tree, tpr_tree, label='Decision Tree (AUC = %0.2f)' % auc_tree)
plt.plot(fpr_forest, tpr_forest, label='Random Forest (AUC = %0.2f)' % auc_forest)
plt.plot(fpr_knn, tpr_knn, label='KNN (AUC = %0.2f)' % auc_knn)
plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()


ValueError: multiclass format is not supported

In [ ]:
# Fit and predict using decision tree model
decision_tree.fit(X_train, y_train)
y_proba_tree = decision_tree.predict_proba(X_test)
y_pred_tree = y_proba_tree[:, 1]
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_pred_tree)
# Fit and predict using random forest model
random_forest.fit(X_train, y_train)
y_proba_forest = random_forest.predict_proba(X_test)
y_pred_forest = y_proba_forest[:, 1]
fpr_forest, tpr_forest, _ = roc_curve(y_test, y_pred_forest)

# Fit and predict using k-nearest neighbors model
knn.fit(X_train, y_train)
y_proba_knn = knn.predict_proba(X_test)
y_pred_knn = y_proba_knn[:, 1]
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred_knn)

Un taux d'exactitude (accuracy) de 24,46% indique que le modèle n'est pas performant.
Cela signifie que seulement 24,46% des prédictions sont correctes.

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

from sklearn.multiclass import OneVsRestClassifier
# Appliquer la méthode OVA pour résoudre le problème multiclasse
ova_model1 = OneVsRestClassifier(decision_tree)
ova_model1.fit(X_train, y_train)
ova_model2 = OneVsRestClassifier(random_forest)
ova_model2.fit(X_train, y_train)
ova_model3 = OneVsRestClassifier(knn)
ova_model3.fit(X_train, y_train)
# Faire des prédictions sur le jeu de test
y_pred_tree = ova_model1.predict(X_test)
y_pred_forest = ova_model2.predict(X_test)
y_pred_knn = ova_model3.predict(X_test)



# Calculer l'aire sous la courbe ROC pour chaque modèle
roc_auc_tree = auc(fpr_tree, tpr_tree)
roc_auc_forest = auc(fpr_forest, tpr_forest)
roc_auc_knn = auc(fpr_knn, tpr_knn)

# Tracer la courbe ROC pour chaque modèle
plt.plot(fpr_tree, tpr_tree, color='darkorange', label='Arbre de décision (AUC = %0.2f)' % roc_auc_tree)
plt.plot(fpr_forest, tpr_forest, color='blue', label='Forêt aléatoire (AUC = %0.2f)' % roc_auc_forest)
plt.plot(fpr_knn, tpr_knn, color='green', label='KNN (AUC = %0.2f)' % roc_auc_knn)
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux positifs')
plt.ylabel('Taux de vrais positifs')
plt.title('Courbe ROC')
plt.legend(loc="lower right")
plt.show()

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
# Appliquer la méthode OVA pour résoudre le problème multiclasse
ova_model1 = OneVsRestClassifier(decision_tree)
ova_model1.fit(X_train, y_train)
ova_model2 = OneVsRestClassifier(random_forest)
ova_model2.fit(X_train, y_train)
ova_model3 = OneVsRestClassifier(knn)
ova_model3.fit(X_train, y_train)
# Faire des prédictions sur le jeu de test
y_pred_tree = ova_model1.predict(X_test)
y_pred_forest = ova_model2.predict(X_test)
y_pred_knn = ova_model3.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

# Calculer les probabilités de chaque classe pour chaque modèle
y_proba_tree = clf_tree.predict_proba(X_test)
y_proba_forest = clf_forest.predict_proba(X_test)
y_proba_knn = clf_knn.predict_proba(X_test)

# Calculer le score AUC pour chaque modèle
roc_auc_tree = roc_auc_score(y_test, y_proba_tree, multi_class='ovr')
roc_auc_forest = roc_auc_score(y_test, y_proba_forest, multi_class='ovr')
roc_auc_knn = roc_auc_score(y_test, y_proba_knn, multi_class='ovr')

In [ ]:
# Calculer les taux de faux positifs et vrais positifs pour le modèle
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_pred_tree)
# Calculer les taux de faux positifs et vrais positifs pour le modèle
fpr_forest, tpr_forest, _ = roc_curve(y_test, y_pred_forest)
# Calculer les taux de faux positifs et vrais positifs pour le modèle
fpr_knn, tpr_knn, _ = roc_curve(y_test, y_pred_knn)

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt

# Fit and predict using decision tree model
clf_tree.fit(X_train, y_train)
y_proba_tree = clf_tree.predict_proba(X_test)
y_pred_tree = y_proba_tree[:, 1]
fpr_tree, tpr_tree, _ = roc_curve(y_test, y_pred_tree)